## 1.Environment Setup

In [ ]:
import os
import json
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold, mutual_info_classif
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer, MinMaxScaler

# Đặt seed để đảm bảo nhất quán giữa các lần chạy
os.environ["PYTHONHASHSEED"] = "42"
np.random.seed(42)
random.seed(42)
print("Seed: 42")

## 2.Data Processing

In [ ]:
# cleveland.csv
# https://drive.google.com/file/d/16HPyuXWXPptt5g3xvS_kR_wXAfjpR1Ju/view?usp=sharing
# !gdown 16HPyuXWXPptt5g3xvS_kR_wXAfjpR1Ju

Meaning of each column data

In [ ]:
DATA_PATH = 'cleveland.csv'
COLUMNS = ['age','sex','cp','trestbps','chol','fbs','restecg',
           'thalach','exang','oldpeak','slope','ca','thal','target']

raw = pd.read_csv(DATA_PATH, header=None)
raw.columns = COLUMNS

for c in ['age','trestbps','chol','thalach','oldpeak','ca','thal']:
    raw[c] = pd.to_numeric(raw[c], errors='coerce')

raw['target'] = (raw['target'] > 0).astype(int)
print("Shape:", raw.shape)
display(raw.head())
display(raw.isna().sum())

In [ ]:
# basic statistic
display(raw.describe())

# Define numerical, categorical columns before plotting
numeric_cols = ['age','trestbps','chol','thalach','oldpeak']
categorical_cols = ['sex','cp','fbs','restecg','exang','slope','ca','thal']

# Plot histograms for numerical columns
raw[numeric_cols].hist(figsize=(10, 8))
plt.tight_layout()
plt.show()

# Plot count plots for categorical columns
for col in categorical_cols:
    plt.figure(figsize=(6, 6))
    raw.groupby([col, 'target']).size().unstack().plot(kind='bar', stacked=True, ax=plt.gca())
    plt.title(f'Distribution of Target by {col}')
    plt.xlabel(col)
    plt.ylabel('Count')
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.show()

## 3.Visual Explorations

In [ ]:
X = raw.drop(columns=['target'])
y = raw['target']

# Colors by target
colors = ['tab:blue' if t==0 else 'tab:red' for t in y]

def scatter_with_regression(x_series, y_series, x_label, y_label, title):
    # Drop NaNs for fitting
    m = x_series.notna() & y_series.notna()
    x = x_series[m].values.reshape(-1, 1)
    yy = y_series[m].values
    # Fit linear regression
    if len(x) > 1:
        model = LinearRegression()
        model.fit(x, yy)
        # Create line
        x_line = np.linspace(x.min(), x.max(), 100).reshape(-1,1)
        y_pred = model.predict(x_line)
    else:
        x_line, y_pred = None, None

    # Plot scatter
    plt.figure()
    plt.scatter(x_series, y_series, c=colors, alpha=0.7)
    plt.xlabel(x_label); plt.ylabel(y_label)
    plt.title(title)

    # Plot regression line
    if x_line is not None:
        plt.plot(x_line.ravel(), y_pred)
    plt.tight_layout()
    plt.show()

# Scatter + regression for selected pairs
if 'age' in X.columns:
    for col in ['chol', 'trestbps', 'thalach']:
        if col in X.columns:
            scatter_with_regression(X['age'], X[col], 'Age', col, f'{col} vs Age (+ regression line)')

# Group means by age bins
if 'age' in X.columns:
    age_bins = pd.cut(X['age'], bins=5)
    for m in ['chol','trestbps','thalach']:
        if m in X.columns:
            plt.figure()
            avg_vals = X.groupby(age_bins)[m].mean()
            plt.plot(range(len(avg_vals)), avg_vals, marker='o', label=f'Average {m}')
            plt.xticks(range(len(avg_vals)), [str(i) for i in avg_vals.index], rotation=45, ha='right')
            plt.xlabel('Age group'); plt.ylabel(m)
            plt.title(f'{m} average per age group')
            plt.legend(); plt.tight_layout(); plt.show()

# Correlation with age
num_df = X[['age','chol','trestbps','thalach']].copy()
corr = num_df.corr(numeric_only=True)['age'].drop('age')
plt.figure()
corr.plot(kind='bar')
plt.ylabel('Correlation with age')
plt.title('Correlation of features with Age')
plt.tight_layout()
plt.show()

## 4.Data Post-processing

🤔**What is data post-processing?**


### 4.1 Raw Dataset

In [ ]:
TARGET = 'target'
raw_feature_cols = [c for c in raw.columns if c != TARGET]
K = len(raw_feature_cols)

X_all = raw[raw_feature_cols]
y_all = raw[TARGET]

cat_proc = Pipeline(steps=[
    ###
    ###
])
num_proc = Pipeline(steps=[
    ###
    ###
])
preprocess = ColumnTransformer([
    ###
    ###
])
raw_pipeline = Pipeline([
    ###
])

X_train, X_temp, y_train, y_temp = train_test_split(
    X_all, y_all, test_size=0.2, stratify=y_all, random_state=42
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
)

X_raw_train = raw_pipeline.fit_transform(X_train, y_train)
X_raw_val = raw_pipeline.transform(X_val)
X_raw_test = raw_pipeline.transform(X_test)

# Get the feature names after preprocessing
preprocessed_feature_names = []
for name, transformer, columns in preprocess.transformers_:
    if hasattr(transformer, 'get_feature_names_out'):
        preprocessed_feature_names.extend(transformer.get_feature_names_out(columns))
    else:
        preprocessed_feature_names.extend(columns)

# Convert NumPy arrays to DataFrames
X_raw_train_df = pd.DataFrame(X_raw_train, columns=preprocessed_feature_names, index=X_train.index)
X_raw_val_df = pd.DataFrame(X_raw_val, columns=preprocessed_feature_names, index=X_val.index)
X_raw_test_df = pd.DataFrame(X_raw_test, columns=preprocessed_feature_names, index=X_test.index)


out_dir = Path('splits'); out_dir.mkdir(parents=True, exist_ok=True)
pd.concat([X_raw_train_df, y_train.rename(TARGET)], axis=1).to_csv(out_dir / 'raw_train.csv', index=False)
pd.concat([X_raw_val_df, y_val.rename(TARGET)], axis=1).to_csv(out_dir / 'raw_val.csv', index=False)
pd.concat([X_raw_test_df, y_test.rename(TARGET)], axis=1).to_csv(out_dir / 'raw_test.csv', index=False)
print("Saved RAW splits. K (RAW features) =", K)

display(X_raw_train_df)
display(X_raw_train_df.isna().sum())

### 4.2 Feature Engineering Dataset

In [ ]:
def add_new_features(Xdf):
    Xdf = Xdf.copy()
    if set(['chol','age']).issubset(Xdf.columns):
        Xdf['chol_per_age'] = Xdf['chol'] / Xdf['age']
    if set(['trestbps','age']).issubset(Xdf.columns):
        Xdf['bps_per_age'] = Xdf['trestbps'] / Xdf['age']
    if set(['thalach','age']).issubset(Xdf.columns):
        Xdf['hr_ratio'] = Xdf['thalach'] / Xdf['age']
    if 'age' in Xdf.columns:
        Xdf['age_bin'] = pd.cut(Xdf['age'], bins=5, labels=False).astype('category')
    return Xdf

generated_cats = ['age_bin']
generated_nums = ['chol_per_age','bps_per_age','hr_ratio']
all_cats = categorical_cols + generated_cats
all_nums = numeric_cols + generated_nums

cat_proc = Pipeline(steps=[
    ###
    ###
])
num_proc = Pipeline(steps=[
    ###
    ###
])

preprocess = ColumnTransformer([
    ###
    ###
])

feature_adder = FunctionTransformer(add_new_features, validate=False)

fe_pipeline = Pipeline([
    ###
    ###
    ###
])

Xt_tr_np = fe_pipeline.fit_transform(X_train, y_train)
Xt_va_np = fe_pipeline.transform(X_val)
Xt_te_np = fe_pipeline.transform(X_test)

ohe = fe_pipeline.named_steps['preprocess'].named_transformers_['cat'].named_steps['onehot']
cat_names = []
for col, cats in zip(all_cats, ohe.categories_):
    for c in cats:
        cat_names.append(f"{col}_{c}")
pre_names = list(all_nums) + cat_names
mask = fe_pipeline.named_steps['var_filter'].get_support()
eng_cols = [name for name, keep in zip(pre_names, mask) if keep]

Xt_tr = pd.DataFrame(Xt_tr_np, columns=eng_cols, index=X_train.index)
Xt_va = pd.DataFrame(Xt_va_np, columns=eng_cols, index=X_val.index)
Xt_te = pd.DataFrame(Xt_te_np, columns=eng_cols, index=X_test.index)

is_discrete = np.array([col.startswith('age_bin_') or any(col.startswith(pfx) for pfx in ['sex_','cp_','fbs_','restecg_','exang_','slope_','ca_','thal_']) for col in Xt_tr.columns], dtype=bool)
mi_scores = mutual_info_classif(Xt_tr.values, y_train.values, discrete_features=is_discrete, random_state=42)
mi_series = pd.Series(mi_scores, index=Xt_tr.columns).sort_values(ascending=False)

# Visualize Top MI
topN = int(min(20, len(mi_series)))
top_mi = mi_series.head(topN).iloc[::-1]
plt.figure(figsize=(10, max(6, 0.35*topN)))
plt.barh(top_mi.index, top_mi.values)
plt.title('Top MI scores (Train)')
plt.xlabel('MI score'); plt.ylabel('Feature')
plt.tight_layout(); plt.show()

K = len([c for c in raw.columns if c != 'target'])
topk_cols = list(mi_series.head(K).index)
print("Selected TOP-K engineered features (K = RAW features):", K)
display(pd.Series(topk_cols, name="fe_topk_features").reset_index(drop=True))

fe_tr = Xt_tr[topk_cols].copy()
fe_tr['target'] = y_train.values
fe_va = Xt_va[topk_cols].copy()
fe_va['target'] = y_val.values
fe_te = Xt_te[topk_cols].copy()
fe_te['target'] = y_test.values

out_dir = Path('splits')
out_dir.mkdir(parents=True, exist_ok=True)
fe_tr.to_csv(out_dir / 'fe_train.csv', index=False)
fe_va.to_csv(out_dir / 'fe_val.csv', index=False)
fe_te.to_csv(out_dir / 'fe_test.csv', index=False)

print("Saved FE splits. K (FE features) =", K)

In [ ]:
display(fe_tr.head())
display(fe_tr.isna().sum())

In [ ]:
# zip folder
!zip -r dataset.zip splits